In [1]:
import os

import numpy as np
import xarray as xr
from pyrte_rrtmgp.rrtmgp_gas_optics import GasOpticsFiles, load_gas_optics
from pyrte_rrtmgp.rrtmgp_data import download_rrtmgp_data
from pyrte_rrtmgp.rte_solver import RTESolver
from pyrte_rrtmgp.all_skys_funcs import compute_profiles, create_gas_dataset, compute_clouds, compute_cloud_optics, expand_to_2d, combine_optical_props


rte_rrtmgp_dir = download_rrtmgp_data()

ncol = 24
nlay = 72
# Compute profiles
p_lay, t_lay, p_lev, t_lev, q, o3 = compute_profiles(300, ncol, nlay)

lw_no_aerosol = f"{rte_rrtmgp_dir}/examples/all-sky/reference/rrtmgp-allsky-lw-no-aerosols.nc"
ref_data = xr.load_dataset(lw_no_aerosol, decode_cf=False)

# Create xarray dataset with profiles
profiles = xr.Dataset(
    data_vars={
        "pres_layer": (["site", "layer"], p_lay),
        "temp_layer": (["site", "layer"], t_lay), 
        "pres_level": (["site", "level"], p_lev),
        "temp_level": (["site", "level"], t_lev),
        "water_vapor": (["site", "layer"], q),
        "ozone": (["site", "layer"], o3)
    }
)


gas_optics_lw = load_gas_optics(gas_optics_file=GasOpticsFiles.LW_G256)
lw_clouds = os.path.join(rte_rrtmgp_dir, "rrtmgp-clouds-lw-bnd.nc")

# for sw
# sfc_alb_dir = 0.06
# sfc_alb_dif = 0.06
# mu0 = 0.86


# Create dataset with gas concentrations
gas_values = {
    'carbon_dioxide_GM': 348e-6,  # scalar
    'methane_GM': 1650e-9,  # scalar
    'nitrous_oxide_GM': 306e-9,  # scalar
    'nitrogen_GM': 0.7808,  # scalar
    'oxygen_GM': 0.2095,  # scalar
    'carbon_monoxide_GM': 0.0  # scalar
}

gases = create_gas_dataset(gas_values, ncol, nlay)

atmosphere = xr.merge([profiles, gases])
top_at_1 = atmosphere["pres_layer"].values[0,0]<atmosphere["pres_layer"].values[0,-1]

t_sfc = t_lev[:, nlay if top_at_1 else 0]
atmosphere["surface_temperature"] = xr.DataArray(t_sfc, dims=["site"])

gas_optics_lw = load_gas_optics(gas_optics_file=GasOpticsFiles.LW_G256)
clear_sky_optical_props = gas_optics_lw.gas_optics.compute(atmosphere, problem_type="absorption", add_to_input=False)

cloud_optics = xr.load_dataset(lw_clouds)

t_sfc = t_lev[0, nlay if top_at_1 else 0]
emis_sfc = expand_to_2d(0.98, ncol, nlay+1, name="surface_emissivity")


cloud_properties = compute_clouds(cloud_optics, ncol, nlay, p_lay, t_lay)

tau = compute_cloud_optics(cloud_properties, cloud_optics)

# Create dataset with optical properties
clouds_optical_props = xr.Dataset(
    {
        "tau": (["site", "layer", "gpt"], tau),
    }
)

clear_sky_optical_props["surface_emissivity"] = 0.98

# Increment the optical properties
combine_optical_props(clouds_optical_props, clear_sky_optical_props)

solver = RTESolver()
fluxes = solver.solve(clear_sky_optical_props, add_to_input=False)


lw_no_aerosol = f"{rte_rrtmgp_dir}/examples/all-sky/reference/rrtmgp-allsky-lw-no-aerosols.nc"
ref_data = xr.load_dataset(lw_no_aerosol, decode_cf=False)

# Assertions
assert np.isclose(fluxes["lw_flux_up"].values, ref_data["lw_flux_up"].values.T, atol=1e-7).all()
assert np.isclose(fluxes["lw_flux_down"].values, ref_data["lw_flux_dn"].values.T, atol=1e-7).all()



In [1]:
import os

import numpy as np
import xarray as xr
from pyrte_rrtmgp.rrtmgp_gas_optics import GasOpticsFiles, load_gas_optics
from pyrte_rrtmgp.rrtmgp_data import download_rrtmgp_data
from pyrte_rrtmgp.rte_solver import RTESolver
from pyrte_rrtmgp.all_skys_funcs import compute_profiles, create_gas_dataset, compute_clouds, compute_cloud_optics, expand_to_2d, combine_optical_props


rte_rrtmgp_dir = download_rrtmgp_data()

ncol = 24
nlay = 72
# Compute profiles
p_lay, t_lay, p_lev, t_lev, q, o3 = compute_profiles(300, ncol, nlay)

sw_no_aerosol = f"{rte_rrtmgp_dir}/examples/all-sky/reference/rrtmgp-allsky-sw-no-aerosols.nc"
ref_data = xr.load_dataset(sw_no_aerosol, decode_cf=False)

# Create xarray dataset with profiles
profiles = xr.Dataset(
    data_vars={
        "pres_layer": (["site", "layer"], p_lay),
        "temp_layer": (["site", "layer"], t_lay), 
        "pres_level": (["site", "level"], p_lev),
        "temp_level": (["site", "level"], t_lev),
        "water_vapor": (["site", "layer"], q),
        "ozone": (["site", "layer"], o3)
    }
)

gas_optics_sw = load_gas_optics(gas_optics_file=GasOpticsFiles.SW_G224)
sw_clouds = os.path.join(rte_rrtmgp_dir, "rrtmgp-clouds-sw-bnd.nc")

# Create dataset with gas concentrations
gas_values = {
    'carbon_dioxide_GM': 348e-6,  # scalar
    'methane_GM': 1650e-9,  # scalar
    'nitrous_oxide_GM': 306e-9,  # scalar
    'nitrogen_GM': 0.7808,  # scalar
    'oxygen_GM': 0.2095,  # scalar
    'carbon_monoxide_GM': 0.0  # scalar
}

gases = create_gas_dataset(gas_values, ncol, nlay)

atmosphere = xr.merge([profiles, gases])
top_at_1 = atmosphere["pres_layer"].values[0,0]<atmosphere["pres_layer"].values[0,-1]

# For SW calculations
sfc_alb_dir = expand_to_2d(0.06, ncol, nlay+1, name="surface_albedo_direct", dims=["site", "level"])
sfc_alb_dif = expand_to_2d(0.06, ncol, nlay+1, name="surface_albedo_diffuse", dims=["site", "level"]) 
mu0 = expand_to_2d(0.86, ncol, nlay+1, name="mu0", dims=["site", "level"])


atmosphere["solar_zenith_angle"] = mu0
atmosphere["surface_albedo_dir"] = sfc_alb_dir
atmosphere["surface_albedo_dif"] = sfc_alb_dif


clear_sky_optical_props = gas_optics_sw.gas_optics.compute(atmosphere, problem_type="two-stream", add_to_input=False)

cloud_optics = xr.load_dataset(sw_clouds)

cloud_properties = compute_clouds(cloud_optics, ncol, nlay, p_lay, t_lay)

tau, ssa, g = compute_cloud_optics(cloud_properties, cloud_optics, lw=False)

# Create dataset with optical properties
clouds_optical_props = xr.Dataset(
    {
        "ssa": (["site", "layer", "gpt"], ssa),
        "g": (["site", "layer", "gpt"], g),
        "tau": (["site", "layer", "gpt"], tau),
    }
)

# Increment the optical properties
#delta scale
combine_optical_props(clouds_optical_props, clear_sky_optical_props)

solver = RTESolver()
fluxes = solver.solve(clear_sky_optical_props, add_to_input=False)


# Assertions
assert np.isclose(fluxes["sw_flux_up"].values, ref_data["sw_flux_up"].values.T, atol=2).all()
assert np.isclose(fluxes["sw_flux_down"].values, ref_data["sw_flux_dn"].values.T, atol=3).all()



KeyError: "No variable named 'solar_zenith_angle'. Variables on the dataset include ['toa_source', 'gpt', 'tau', 'ssa', 'g', 'surface_albedo_direct', 'surface_albedo_diffuse', 'bnd_limits_gpt']"